<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/porto_seguro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 【行ったこと】

# ▼ 

In [6]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c porto-seguro-safe-driver-prediction

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c porto-seguro-safe-driver-prediction
!ls

# ▼ unzip
!unzip '*.zip'
!ls
# ▲ unzip

# ▼ kaggle notebookの場合
# # !cd ../
# !ls ../input/porto-seguro-safe-driver-prediction

# TRAIN_DATA = '../input/porto-seguro-safe-driver-predicti1on/train.csv'
# df = pd.read_csv(TRAIN_DATA)
# ▲ kaggle notebookの場合

# ▼ 行表示
# ctrl + m + l
# ▲ 行表示

Download 100%.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 83% 25.0M/30.0M [00:00<00:00, 47.7MB/s]
100% 30.0M/30.0M [00:00<00:00, 75.1MB/s]
  0% 0.00/2.12M [00:00<?, ?B/s]
100% 2.12M/2.12M [00:00<00:00, 143MB/s]
 92% 41.0M/44.4M [00:00<00:00, 32.2MB/s]
100% 44.4M/44.4M [00:00<00:00, 81.5MB/s]
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
adc.json  sample_data		     test.csv.zip
drive	  sample_submission.csv.zip  train.csv.zip


In [0]:
import pandas as pd
import numpy as np

from logging import getLogger

TRAIN_DATA = './train.csv'
TEST_DATA =  './test.csv'

logger = getLogger(__name__)

def read_csv(path):
  logger.debug('enter')
  logger.debug('exit')
  return df

def load_train_data():
  logger.debug('enter')
  df = pd.read_csv(TRAIN_DATA)
  logger.debug('exit')
  return df

def load_test_data():
  logger.debug('enter')
  df = pd.read_csv(TEST_DATA)
  logger.debug('exit')
  return df

  if __name__ == '__main__':
    print(load_train_data().head())
    print(load_test_data().head())